In [1]:
import pandas as pd
import numpy as np
import time
from MyNLP import Divider

In [2]:
# データの読みこみ
comment_list = []
comment_sentence_list = []
df1 = pd.read_csv("comment_keitaiso_0~99_2022_09_22.csv", encoding="Shift-jis")
df2 = pd.read_csv("comment_keitaiso_100~199_2022_09_22.csv", encoding="Shift-jis")
df3 = pd.read_csv("comment_keitaiso_200~299_2022_09_22.csv", encoding="Shift-jis")
df4 = pd.read_csv("comment_keitaiso_300~397_2022_09_22.csv", encoding="Shift-jis")
df5 = pd.read_csv("comment_keitaiso_398~459_2022_09_22.csv", encoding="Shift-jis")

comment_sentence_list.extend(df1.to_numpy().tolist())
comment_sentence_list.extend(df2.to_numpy().tolist())
comment_sentence_list.extend(df3.to_numpy().tolist())
comment_sentence_list.extend(df4.to_numpy().tolist())
comment_sentence_list.extend(df5.to_numpy().tolist())
comment_sentence_list = pd.DataFrame(comment_sentence_list)

# コメントの列だけを抽出
comment_sentence_list = comment_sentence_list.iloc[:, 1]

# 欠損値のデータを削除
comment_sentence_list = comment_sentence_list.dropna()
# dfをリストに変換
comment_sentence_list = comment_sentence_list.tolist()

In [3]:
len(comment_sentence_list)

1052

In [4]:
import torch
from transformers import BertModel, BertTokenizer

# GPUが利用可能な場合はGPUを、利用できない場合はCPUを使用する
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BERTの事前学習済みモデルを読み込む
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# モデルをデバイスに転送する
model.to(device)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [5]:
# テキストデータをトークナイズする
from transformers import BertJapaneseTokenizer

# tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
# comment_sentence_list = ["代わりに自分は原則としてヨドバシしか利用していない。"]
inputs = tokenizer(comment_sentence_list, return_tensors="pt", padding=True).to(device)

In [6]:
# モデルに入力を与えて、テキストデータをエンコードする

input_size = 10
outputs = []
for i in range(0, len(comment_sentence_list), input_size):
    input_dict = {'input_ids':inputs['input_ids'][i:i+input_size], 'token_type_ids':inputs['token_type_ids'][i:i+input_size], 'attention_mask':inputs['attention_mask'][i:i+input_size]}
    encoded_outputs = model(**input_dict)
    outputs.extend(encoded_outputs.last_hidden_state.to('cpu').detach().numpy().copy())

In [11]:
# 単語分散表現のリストを文章の分散表現で表す（単語ベクトルの平均を求める）
sentence_vectors = []

for word_vectors in outputs:
    sentence_vector = np.mean(word_vectors, axis=0)
    sentence_vectors.append(sentence_vector)

[[-0.12431185  0.45324123 -0.01948236 ... -0.11886275 -0.1126607
   0.01430907]
 [-0.3944146   0.31560507  0.513338   ...  0.3272531  -0.2890068
   0.13553962]
 [-0.2461678  -0.2534163   0.10546274 ...  0.19331867 -0.72002935
   0.4278174 ]
 ...
 [ 0.14662236  0.1189343  -0.10962641 ...  0.07063868 -0.07098422
   0.38611576]
 [ 0.03433736  0.3717186   0.07763825 ... -0.0037215  -0.15763126
   0.34569058]
 [-0.01838047  0.51830405  0.59891856 ...  0.1731238  -0.46348763
   0.41143265]]
[[-0.14199841  0.65827084 -0.4702268  ... -0.3027515  -0.05917313
  -0.21978901]
 [-0.15290694  0.7314866   0.09423125 ... -0.16343576 -0.0821094
  -0.06166288]
 [-0.18775262  0.3516099  -0.20585538 ... -0.36284924  0.00230647
   0.05828978]
 ...
 [ 0.03222015 -0.1367613  -0.55976564 ... -0.29363737  0.46949473
  -0.10106376]
 [-0.14491531  0.00391832  0.07956743 ... -0.52600455  0.21185218
   0.39263833]
 [-0.23466235  0.5039982   0.43851873 ... -0.60272396  0.12410633
  -0.06221094]]
[[-0.35639858  0.47

In [17]:
# コメントの類似度を総当たりで計算するプログラム

from sklearn.metrics.pairwise import cosine_similarity

pair_list = []

for id1, comment1 in enumerate(sentence_vectors):
    for id2, comment2 in enumerate(sentence_vectors):
        # 同じコメント同士でなければ、コサイン類似度を計算する
        if not id1 == id2: 
            similarity = cosine_similarity(comment1.reshape(1, -1), comment2.reshape(1, -1))
            if not [id2, id1, comment_sentence_list[id2], comment_sentence_list[id1], similarity[0][0]] in pair_list:
                pair_list.append([id1, id2, comment_sentence_list[id1], comment_sentence_list[id2], similarity[0][0]])
    print(id1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
# CSVに保存するプログラム
import csv
with open('Result/2023_04_16/comment_similarity_Bert_word_tf-idf_vector.csv', 'w', newline='') as csv_file:
    fieldnames = ['id1','id2', 'comment1', 'comment2', 'similarity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for pair in pair_list:
        data = {'id1': pair[0],'id2': pair[1], 'comment1': pair[2], 'comment2': pair[3], 'similarity': pair[4]}
        try:
            writer.writerow(data)
        except UnicodeEncodeError:
            continue